# Arabic Characters Recognition Using Convolutional Neural Network

## Overview

This project focuses on training a Convolutional Neural Network (CNN) to recognize Arabic handwritten characters. Handwritten character recognition is a crucial task in optical character recognition (OCR), with applications in document digitization, automated data entry, and assistive technologies.

Arabic script poses unique challenges due to its cursive nature, varying character shapes based on position in a word, and high inter-class similarity. To tackle this, we utilize deep learning, leveraging CNNs for their ability to automatically extract features from images and learn hierarchical patterns.

In this notebook, I will:

Load and preprocess an Arabic handwritten character dataset.
Build and train a CNN model for classification.
Evaluate performance using accuracy and loss metrics.
Analyse and tune the hyperparameters to build the best model possible.
Visualize training process using TensorBoard
This project aims to improve handwritten Arabic text recognition, demonstrating the power of deep learning in solving real-world OCR challenges

## Training the basic model with one convolutional layer and tuning hyperparameters:

Data was taken from Kaggle:  https://www.kaggle.com/datasets/mloey1/ahcd1

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import datetime

# Importing data
train_images = pd.read_csv('archive/csvTrainImages 13440x1024.csv', header=None)
train_labels = pd.read_csv('archive/csvTrainLabel 13440x1.csv', header=None)
test_images = pd.read_csv('archive/csvTestImages 3360x1024.csv', header=None)
test_labels = pd.read_csv('archive/csvTestLabel 3360x1.csv', header=None)

# Data preparation
train_images = train_images.values.reshape(-1, 32, 32, 1).astype('float32')
test_images = test_images.values.reshape(-1, 32, 32, 1).astype('float32')

train_labels -= 1
test_labels -= 1


num_classes = 28 
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Splitting data into train/test
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Function for building and training the model
def build_and_train_model(conv_layers=1, filters=32, kernel_size=(3, 3), padding='valid', strides=(1, 1),
                          use_batch_norm=False, dropout_rate=0.0):
    model = Sequential()

    
    for _ in range(conv_layers):
        model.add(Conv2D(filters, kernel_size=kernel_size, padding=padding, strides=strides, activation='relu', input_shape=(32, 32, 1)))
        if use_batch_norm:
            model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # TensorBoard log
    log_dir = "logs/fit/" + "conv_layers" + str(conv_layers) + "use_batch_norm=" + str(use_batch_norm) + "dropout_rate" + str(dropout_rate) + "padding" + str(padding) + "strides" + str(strides)
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Training a base model with 1 convolutional layer
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64, callbacks=[tensorboard_callback])

    return model, history

# Experiment 1: Different padding and strides
params = [
    {'padding': 'valid', 'strides': (1, 1)},
    {'padding': 'same', 'strides': (1, 1)},
    {'padding': 'same', 'strides': (2, 2)},
]

for param in params:
    print(f"Experiment with padding={param['padding']} and strides={param['strides']}")
    model, history = build_and_train_model(padding=param['padding'], strides=param['strides'])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f'Validation accuracy: {val_acc:.4f}\n')

# Experiment 2: Alternative structures with different numbers of convolutional layers, normalisation and dropout
architectures = [
    {'conv_layers': 2, 'use_batch_norm': False, 'dropout_rate': 0.0},
    {'conv_layers': 3, 'use_batch_norm': True, 'dropout_rate': 0.2},
    {'conv_layers': 3, 'use_batch_norm': True, 'dropout_rate': 0.5},
]

for arch in architectures:
    print(f"Experiment with conv_layers={arch['conv_layers']}, use_batch_norm={arch['use_batch_norm']}, dropout_rate={arch['dropout_rate']}")
    model, history = build_and_train_model(conv_layers=arch['conv_layers'], use_batch_norm=arch['use_batch_norm'], dropout_rate=arch['dropout_rate'])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f'Validation accuracy: {val_acc:.4f}\n')



Experiment with padding=valid and strides=(1, 1)
Epoch 1/10


2025-03-17 13:09:58.163428: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


168/168 [==============================] - 2s 12ms/step - loss: 3.5405 - accuracy: 0.1984 - val_loss: 2.2645 - val_accuracy: 0.3188
Epoch 2/10
168/168 [==============================] - 2s 12ms/step - loss: 1.6643 - accuracy: 0.4798 - val_loss: 1.3366 - val_accuracy: 0.5781
Epoch 3/10
168/168 [==============================] - 2s 12ms/step - loss: 0.9503 - accuracy: 0.6837 - val_loss: 0.9797 - val_accuracy: 0.6853
Epoch 4/10
168/168 [==============================] - 2s 14ms/step - loss: 0.5659 - accuracy: 0.8108 - val_loss: 0.8989 - val_accuracy: 0.7314
Epoch 5/10
168/168 [==============================] - 2s 12ms/step - loss: 0.3624 - accuracy: 0.8770 - val_loss: 0.8826 - val_accuracy: 0.7500
Epoch 6/10
168/168 [==============================] - 2s 14ms/step - loss: 0.2515 - accuracy: 0.9182 - val_loss: 0.8609 - val_accuracy: 0.7705
Epoch 7/10
168/168 [==============================] - 2s 14ms/step - loss: 0.1669 - accuracy: 0.9462 - val_loss: 0.8850 - val_accuracy: 0.7656
Epoch 8/10

We can see that the best result in terms of metrics is the one with padding=same, strides=(1,1). This model has an accuracy of roughly 78%.
Also after analysis of parameters and convolutional numbers analysis we can see that best parameters are 3 conv layers, 'use_batch_norm': True, 'dropout_rate': 0.2

Let's train such model and check the results for each class (letter):

In [2]:
!rm -rf ./logs/

In [3]:
best_model = build_and_train_model(conv_layers=3, use_batch_norm=True, dropout_rate=0.2, padding='same', strides=(1,1))[0]

# Evalutating
test_loss, test_acc = best_model.evaluate(test_images, test_labels, verbose=0)
print(f'Test accuracy: {test_acc:.4f}')

# Getting metrics
y_pred = best_model.predict(test_images)
y_true = np.argmax(test_labels, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes, digits=4))
print(f"AUC: {roc_auc_score(test_labels, y_pred, multi_class='ovr'):.4f}")

Epoch 1/10
168/168 [==============================] - 5s 26ms/step - loss: 1.9282 - accuracy: 0.4341 - val_loss: 0.8810 - val_accuracy: 0.7251
Epoch 2/10
168/168 [==============================] - 4s 25ms/step - loss: 0.6772 - accuracy: 0.7797 - val_loss: 0.4955 - val_accuracy: 0.8445
Epoch 3/10
168/168 [==============================] - 4s 26ms/step - loss: 0.3931 - accuracy: 0.8713 - val_loss: 0.3680 - val_accuracy: 0.8713
Epoch 4/10
168/168 [==============================] - 4s 27ms/step - loss: 0.2663 - accuracy: 0.9146 - val_loss: 0.2539 - val_accuracy: 0.9156
Epoch 5/10
168/168 [==============================] - 5s 29ms/step - loss: 0.1984 - accuracy: 0.9354 - val_loss: 0.2450 - val_accuracy: 0.9226
Epoch 6/10
168/168 [==============================] - 5s 28ms/step - loss: 0.1570 - accuracy: 0.9488 - val_loss: 0.2580 - val_accuracy: 0.9163
Epoch 7/10
168/168 [==============================] - 5s 29ms/step - loss: 0.1215 - accuracy: 0.9610 - val_loss: 0.2424 - val_accuracy: 0.9189

For all 28 classes we've got great metrics - good result

In [4]:
%load_ext tensorboard

### Visualisation of training in TensorBoard

In [5]:
%tensorboard --logdir logs/fit

Launching TensorBoard...